In [63]:
from soma import aims
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [127]:
%matplotlib inline

In [64]:
#import anatomist.direct.api as ana
#a = ana.Anatomist()

In [65]:
path_mean_curv = Path("/neurospin/tmp/tsanchez/mean_curvature_ammon.nii.gz")
obj = aims.read(str(path_mean_curv))
obj.np.shape

(256, 256, 124, 1)

In [66]:
path_mean_curv.parent

PosixPath('/neurospin/tmp/tsanchez')

In [67]:
#win = a.createWindow("Axial")
#view_anatomist = a.toAObject(obj) 
#win.addObjects(view_anatomist)

In [68]:
val = obj.np

To retrieve the differents parts of the cerebellum : 
- White matter : -1
- Sulci : 1
- 0 : The rest

In [69]:
WM_RANGE = -0.3967
SULCI_RANGE = 0.464


In [70]:
# Mask for WM and Sulci
WM_mask = (val <= WM_RANGE) 
SULCI_mask = (val >= SULCI_RANGE)
Other_mask = (val > WM_RANGE) & (val < SULCI_RANGE) 

In [71]:
# Changing the values according to what is mentionned above
val[WM_mask] = -1
val[SULCI_mask] = 1
val[Other_mask] = 0
np.unique(val, return_counts=True)

(array([-1.,  0.,  1.], dtype=float32), array([ 737904, 6760262,  628298]))

In [72]:
filename = path_mean_curv.parent / "threshold_mean_curve_ammon.nii.gz"
filename

PosixPath('/neurospin/tmp/tsanchez/threshold_mean_curve_ammon.nii.gz')

In [73]:
aims.write(obj, str(filename))

## On HCP
Subjects :


Commande pour produire les fichiers de mean_curvatures : 
`VipGeometry -i <input_path> -m mc -o <output_path>`  

Few exemples are in the `/neurospin/tmp/tsanchez` folder to check if the threshold is right

In [77]:
path_data = Path("/neurospin/tmp/tsanchez")
path_hcp = path_data / "hcp" 
path_pclean = path_data / "pclean" 
path_hcp, path_pclean

(PosixPath('/neurospin/tmp/tsanchez/hcp'),
 PosixPath('/neurospin/tmp/tsanchez/pclean'))

In [78]:
subjects_hcp = [
    "163331", 
    "518746", 
    "991267", 
]

subjects_pclean = [
    "s12158", 
    "s12401", 
    "s12635", 
]

path_subjects_hcp = [path_hcp / "mean_curvature" / f"mean_curvature_{subject}.nii.gz" for subject in subjects_hcp]
path_subjects_pclean = [path_pclean /"mean_curvature" / f"mean_curvature_{subject}.nii.gz" for subject in subjects_pclean]

path_subjects_hcp, path_subjects_pclean

([PosixPath('/neurospin/tmp/tsanchez/hcp/mean_curvature/mean_curvature_163331.nii.gz'),
  PosixPath('/neurospin/tmp/tsanchez/hcp/mean_curvature/mean_curvature_518746.nii.gz'),
  PosixPath('/neurospin/tmp/tsanchez/hcp/mean_curvature/mean_curvature_991267.nii.gz')],
 [PosixPath('/neurospin/tmp/tsanchez/pclean/mean_curvature/mean_curvature_s12158.nii.gz'),
  PosixPath('/neurospin/tmp/tsanchez/pclean/mean_curvature/mean_curvature_s12401.nii.gz'),
  PosixPath('/neurospin/tmp/tsanchez/pclean/mean_curvature/mean_curvature_s12635.nii.gz')])

In [108]:
def apply_mask(obj, WM_thresh : int, Sulci_thresh : int,saving_path : Path, to_save : bool = False) -> np.ndarray :
    #To numpy 
    vol = obj.np

    # Mask for WM and Sulci
    WM_mask = (vol <= WM_thresh) 
    Sulci_mask = (vol >= Sulci_thresh)
    Other_mask = (vol > WM_thresh) & (vol < Sulci_thresh) 

    #Apply Mask : 
    vol[WM_mask] = -1
    vol[Sulci_mask] = 1
    vol[Other_mask] = 0

    if to_save : 
        print(f"Saving to {saving_path}")
        aims.write(obj, filename=str(saving_path))
    
    return vol


In [109]:
path_subjects_hcp[0].name[-13:-7]

'163331'

In [129]:
# Loading file to dict
dict_hcp = dict([(path.name[-13:-7],aims.read(str(path))) for path in path_subjects_hcp])
dict_pclean = dict([(path.name[-13:-7],aims.read(str(path))) for path in path_subjects_pclean])

In [111]:
hcp_saving_fold = path_hcp / "thresh"
pclean_saving_fold = path_pclean / "thresh"
hcp_saving_fold, pclean_saving_fold

(PosixPath('/neurospin/tmp/tsanchez/hcp/thresh'),
 PosixPath('/neurospin/tmp/tsanchez/pclean/thresh'))

In [113]:
WM_RANGE = -0.3967
SULCI_RANGE = 0.464

hcp_dict_thresh = dict(
    [
        (key, apply_mask(dict_hcp[key],
                         WM_thresh= WM_RANGE,
                         Sulci_thresh=SULCI_RANGE,
                         saving_path= hcp_saving_fold / f"threshold_cerebellum_{key}.nii.gz",
                         to_save=True)) for key in dict_hcp.keys()
    ]
)


pclean_dict_thresh = dict(
    [
        (key, apply_mask(dict_pclean[key],
                         WM_thresh= WM_RANGE,
                         Sulci_thresh=SULCI_RANGE,
                         saving_path= pclean_saving_fold / f"threshold_cerebellum_{key}.nii.gz",
                         to_save=True)) for key in dict_pclean.keys()
    ]
)

Saving to /neurospin/tmp/tsanchez/hcp/thresh/threshold_cerebellum_163331.nii.gz
Saving to /neurospin/tmp/tsanchez/hcp/thresh/threshold_cerebellum_518746.nii.gz
Saving to /neurospin/tmp/tsanchez/hcp/thresh/threshold_cerebellum_991267.nii.gz
Saving to /neurospin/tmp/tsanchez/pclean/thresh/threshold_cerebellum_s12158.nii.gz
Saving to /neurospin/tmp/tsanchez/pclean/thresh/threshold_cerebellum_s12401.nii.gz
Saving to /neurospin/tmp/tsanchez/pclean/thresh/threshold_cerebellum_s12635.nii.gz
